# Colorization
Class project - CS231N - Stanford University

Vincent Billaut  
Matthieu de Rochemonteix  
Marc Thibault  

See our GitHub [repo](https://github.com/vincentbillaut/all-colors-matter) for more details on the implementation.

## Imports

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
from matplotlib.pyplot import imread, imsave
import numpy as np

%matplotlib inline

from matplotlib.pyplot import imread
from matplotlib import animation
from IPython.display import display, HTML
from tqdm import tqdm_notebook

####################################################
# Setting working directory to enable relative paths
import os
os.chdir("../")
import pickle
####################################################

from models.coloringmodel import Config
from models.naive_convnet import NaiveConvColoringModel
from models.unet import UNetColoringModel
from utils.dataset import Dataset
from utils.color_utils import RGB_to_YUV, YUV_to_RGB
from utils.data_utils import load_image_jpg_to_YUV, dump_YUV_image_to_jpg
from utils.color_discretizer import ColorDiscretizer
from utils.data_augmentation import DataAugmenter
from utils.video_utils import smoothen_frame_list

## Load existing model

In [ ]:
# Enter output folder name
output_folder = "outputs/20180604_223338-2cd6/"

In [ ]:
config = Config("configs/config_unet_suncoast2.json")

cd = ColorDiscretizer(max_categories=config.max_categories)
cd_loaded = pickle.load(open("notebooks/cd.pkl", "rb"))
for k in cd_loaded.__dict__:
    cd.__dict__[k] = cd_loaded.__dict__[k]

In [ ]:
config.val_path = "data/long_video_frames_/"

In [ ]:
da = DataAugmenter()
dataset = Dataset(config.train_path, config.val_path, cd, da)
model = UNetColoringModel(config, dataset)

In [ ]:
model.load(output_folder)

In [ ]:
image_paths = [os.path.join(config.val_path, impath) for impath in os.listdir(config.val_path)]
image_paths = sorted(image_paths)

In [ ]:
# specific sort to extract the frame number.
image_paths = sorted(image_paths, key=lambda t: int(t[29:-4]))

In [ ]:
image_paths

### Utility functions

In [ ]:
def colorize_smooth_video_tojpg(image_paths, cd, weight_alpha=.2, temperature=1.):
    n_images = len(image_paths)
    memory_prediction = None
    
    for i in tqdm_notebook(range(n_images)):        
        image_path = image_paths[i]
        loss, pred_image_categories, (im_yscale, im_uvscale, msk) = model.pred_color_one_image(image_path)
        mask_shape = [0, 0]
        if msk[:, 0].mean() == 1.:
            mask_shape[0] = msk.shape[1]
        else:
            mask_shape[0] = np.argmin(msk[:, 0])
        if msk[0, :].mean() == 1.:
            mask_shape[1] = msk.shape[0]
        else:
            mask_shape[1] = np.argmin(msk[0, :])
        
        pred_frame = pred_image_categories[0, :mask_shape[0], :mask_shape[1], :]
        yscale_frame = im_yscale[:mask_shape[0], :mask_shape[1]]
        uvtruth_frame = im_uvscale[:mask_shape[0], :mask_shape[1], :]
        

        true_YUV_image = np.concatenate([yscale_frame, uvtruth_frame], axis=2)
        dump_YUV_image_to_jpg(true_YUV_image, "outputs/video/true_frame{}.png".format(i))
        
        true_greyscale_image = np.concatenate([yscale_frame]*3, axis=2)
        true_greyscale_image = (true_greyscale_image * 255.).astype("uint8")
        imsave(fname="outputs/video/greyscale_frame{}.png".format(i), arr=true_greyscale_image,format = 'png')
        
        pred_UVimage = cd.UVpixels_from_distribution(pred_frame, temperature=temperature)
        predicted_YUV_image = np.concatenate([yscale_frame, pred_UVimage], axis=2)
        dump_YUV_image_to_jpg(predicted_YUV_image, "outputs/video/predicted_frame{}.png".format(i))
        
        if memory_prediction is None:
            memory_prediction = pred_frame
        else:
            memory_prediction *= np.exp(-weight_alpha)
        
        smooth_pred_frame = (memory_prediction + pred_frame) / (1 + np.exp(-weight_alpha))
        pred_UVimage = cd.UVpixels_from_distribution(smooth_pred_frame, temperature=temperature)
        predicted_YUV_image = np.concatenate([yscale_frame, pred_UVimage], axis=2)
        dump_YUV_image_to_jpg(predicted_YUV_image, "outputs/video/predicted_smooth_frame{}.png".format(i))
        
        memory_prediction = smooth_pred_frame
        

In [ ]:
colorize_smooth_video_tojpg(image_paths, cd)

### Smoothing the predictions

In [ ]:
w = np.exp(.2 * np.arange(10))
w = w / sum(w)
filter_size = len(w)
plt.plot(range(-filter_size, 0), w)

In [ ]:
prediction_list_smoothened = smoothen_frame_list(prediction_list, conv_weights=w)

In [ ]:
#dump_pred_videos(prediction_list[filter_size-1:], prediction_list_smoothened, 
#                   yscale_list[filter_size-1:], uvtruth_list[filter_size-1:], cd)